In [11]:
#Read the data into main_data
main_data = {}
with open('data.txt', 'r') as f:
    x = f.read().split()

for i in x:
    if not ("," in i):
        main_data[int(i)] = {}
        current_entry = int(i)
    else:
        connection = i.split(",")
        main_data[current_entry][int(connection[0])] = int(connection[1])

In [37]:
def get_min(current_map, reached):
    copy_map = current_map.copy()
    for key in reached:
        copy_map.pop(key)
    return min(copy_map, key=copy_map.get)

def dijkstra(edge,start=1):
    #construct map
    current_map = {}
    for i in range(1,201):
        current_map[i] = 1000000
    current_map[start] = 0
    
    reached = set()
    while(reached != set(range(1,201))):
        entry = get_min(current_map, reached)
        reached.add(entry)
        for key,value in edge[entry].items():
            if(current_map[key] > current_map[entry] + value):
                current_map[key] = current_map[entry] + value
    
    return current_map

In [39]:
result = dijkstra(main_data, 1)

In [43]:
accessed = set()
min_list = {}
for i in range(0,11):
    entry = get_min(result,accessed)
    accessed.add(entry)
    min_list[entry] = result[entry]
print(min_list)

{1: 0, 114: 508, 140: 546, 92: 647, 145: 648, 129: 676, 70: 743, 9: 745, 199: 815, 65: 826, 80: 982}
